In [1]:
using LinearAlgebra, ToeplitzMatrices, Random, IterativeSolvers, FunctionOperators, Printf
Random.seed!(1);

### IRLS for robust principal component analysis

In [2]:
function RPCA_IRLS(
        Xᴳᵀ::AbstractArray,                     # ground truth for MSE evaluation
        y::AbstractArray,                       # under-sampled data
        Φ::FunctionOperator;                    # sampling operator (P_Omaga in the paper)
        img_size::NTuple = size(Xᴳᵀ),           # size of output matrix
        r̃::Int,                                 # rank estimate of low-rank part
        s̃::Int,                                 # sparsity estimate of sparse part
        λ::Real = 1,                            # regularization param. to balance sparsity and low-rankness
        μ::Real = 1e-3,                         # regularization param. for separation strictness
        δ::Real = 1e-3,                         # smoothing parameter for log (see eq. 5)
        maxIter::Union{Int, Nothing} = nothing, # number of CG iteration steps
        N::Int = 10,                            # number of iterations
        verbose::Bool = false)                  # print rank and loss value in each iteration
    
    # Initialize variables
    dType = eltype(y)
    d₁, d₂ = img_size
    maxIter = maxIter isa Nothing ? max(r̃*(r̃+d₁+d₂), s̃*(s̃+d₁+d₂)) : maxIter
    ϵ = Inf
    X₀ = Φ' * y # that's basically Pᵃ_Omega * P_Omega (M)
    Wₛ, Wₗ = I, I
    ΦᵃΦ = Φ' * Φ
    L, S = copy(X₀), zeros(dType, size(X₀))
    X = L + S
    
    r, c, n, s, e = rank(L, atol = 1e-3), norm(y - Φ * X), norm(Xᴳᵀ - X), svdvals(L)[1], ϵ
    n, c, s, e = @sprintf("%7.3f", n), @sprintf("%7.3f", c), @sprintf("%7.3f", s), @sprintf("%7.3f", e)
    verbose && println("k = 0,\trank(L) = $r,\t‖y - Φ * X‖₂ = $c,\t‖Xᴳᵀ - X‖₂ = $n,\tσ₁ = $s,\tϵ = $e")
    
    for k in 1:N
        
        # Equation under (29)
        Aₗ = reshape((λ/μ*Wₛ + ΦᵃΦ) * Wₗ + λ * Wₛ * ΦᵃΦ, inDims=(d₁*d₂,), outDims=(d₁*d₂,))
        bₗ = λ * Wₛ * X₀
        cg!(vec(L), Aₗ, vec(bₗ), maxiter = maxIter) # solve Aₗ⋅L = bₗ for L
        
        # Equation 30
        Aₛ = reshape(λ/μ*Wₛ + ΦᵃΦ, inDims=(d₁*d₂,), outDims=(d₁*d₂,))
        bₛ = X₀ - ΦᵃΦ * L
        cg!(vec(S), Aₛ, vec(bₛ), maxiter = maxIter) # solve Aₛ⋅S = bₛ for S
        
        F = svd(L)
        U, σ, V = F.U, F.S, F.V
        
        # Equation 32
        Sₛ₊₁ = sort(abs.(vec(S)), rev=true)[s̃+1]
        println("\t\x1b[31m|S|₍ₛ₊₁₎/(2λ) = ", @sprintf("%7.3g", Sₛ₊₁/(2λ)),
            ", σᵣ₊₁ = ", @sprintf("%7.3f", σ[r̃+1]), "\x1b[0m")
        ϵ = min(max(Sₛ₊₁/(2λ), σ[r̃+1]), ϵ)
        
        # Equation 22
        wₛ = [1 / max.(abs(S[i,j]), δ)^2 + δ^2 for i in 1:d₁, j in 1:d₂]
        # Equation 24
        Wₛ = FunctionOperator{dType}(name = "Wₛ", inDims = (d₁, d₂), outDims = (d₁, d₂),
            forw = Z -> wₛ .* Z)
        # Equation 23 and 15 with q = 0 (typo in def of H₁: mean of σ̃ᵢ and σ̃ᵢ ??)
        σ̃ᵢ(i) = 1 / (max(σ[i], ϵ) + ϵ^2)
        d = min(d₁, d₂)
        H₁ = [sqrt(σ̃ᵢ(i) * σ̃ᵢ(j)) for i in 1:d, j in 1:d]
        # Equation 25 (typo: second V⁽ᵏ⁾ should be adjoint)
        Wₗ = FunctionOperator{dType}(name = "Wₗ", inDims = (d₁, d₂), outDims = (d₁, d₂),
            forw = Z -> U * (H₁ .* (U' * Z * V)) * V')
        
        X = L + S

        # Print
        r, c, n, s, e = rank(L, atol = 1e-3), norm(y - Φ * X), norm(Xᴳᵀ - X), σ[1], ϵ
        n, c, s, e = @sprintf("%7.3f", n), @sprintf("%7.3f", c), @sprintf("%7.3f", s), @sprintf("%7.3f", e)
        verbose && println("k = $k,\trank(L) = $r,\t‖y - Φ * X‖₂ = $c,\t‖Xᴳᵀ - X‖₂ = $n,\tσ₁ = $s,\tϵ = $e")
    end
end

RPCA_IRLS (generic function with 1 method)

### Some helper functions

In [3]:
import Base.size
function Base.size(FO::FunctionOperator, d::Int)
    @assert d in [1, 2]
    prod(d == 1 ? FO.outDims : FO.inDims)
end

In [4]:
# This function randomly samples a $(d₁ \times d₂)$ sparse matrix with ones at $m$ randomly chosen
# coordinates (uniform without replacement). The output matrix has at least $r$ non-zero entries
# in each row and each column, where $r$ is a specified positive integer. The number of ones in the
# output matrix is exactly $m$.
function generateΦ(d₁, d₂, r, m)
    @assert max(d₁, d₂) * r ≤ m
    @assert m ≤ d₁ * d₂
    @assert r ≤ d₁
    @assert r ≤ d₂
    
    # generate a square matrix where each row and each column has exactly r ones
    initial = Circulant([fill(1, r)..., fill(0, min(d₁, d₂) - r)...])
    
    # Extend that matrix to a d₁×d₂ matrix where each row and each column has at least r ones
    # That is accomplished by repeating the "initial" matrix and then cropping
    if d₁ < d₂
        M = repeat(initial, outer = (1, ceil(Int, d₂ / d₁)))
    elseif d₁ > d₂
        M = repeat(initial, outer = (ceil(Int, d₁ / d₂), 1))
    else
        M = initial
    end
    M = M[1:d₁, 1:d₂]
    
    # Randomly switch zeros to ones until exactly m number of ones are in the matrix
    zero_places = findall(M .== 0)
    number_of_missing_ones = m - (d₁*d₂ - length(zero_places))
    number_of_missing_ones > 0 && (M[shuffle(zero_places)[1:number_of_missing_ones]] .= 1)
    
    # Then randomize matrix by permutating rows and columns a couple times
    for i in 1:10
        M .= M[shuffle(1:end), :] # shuffle rows
        M .= M[:, shuffle(1:end)] # shuffle columns
    end
    
    M
end

generateΦ (generic function with 1 method)

In [5]:
function maskToMatrix(Φᴹ)
    m = convert(Int, sum(Φᴹ))
    d₁, d₂ = size(Φᴹ)

    Φ = zeros(m, length(Φᴹ))
    non_zero_places = findall(vec(Φᴹ) .== 1)
    for i in 1:m
        Φ[i, non_zero_places[i]] = 1
    end
    return Φ
end

maskToMatrix (generic function with 1 method)

### Generate data

#### That's how Chirstian generated the data to compare algorithms:

In [6]:
d₁, d₂, r, s = 60, 40, 7, 15
df_LR = r * (d₁ + d₂ - r) # Number of degrees of freedom of the setting
m = floor(Int, min(1.05 * df_LR, d₁ * d₂))

dType = ComplexF64

# Generate a matrix with rank = r
U, Σ, V = randn(dType, d₁, r), Diagonal(randn(r)), randn(dType, d₂, r)

# Generate a sparse matrix with exactly s non-zero values
S = zeros(d₁, d₂)
S[randperm(d₁*d₂)[1:s]] = rand(s)

# Ground Truth matrix
Xᴳᵀ = U * Σ * V' + S

@show size(Xᴳᵀ)
@show rank(Xᴳᵀ);

Φᴹ = generateΦ(d₁, d₂, r, m)
Φ = FunctionOperator{dType}(name = "Φ", inDims = (d₁, d₂), outDims = (d₁, d₂),
    forw = (b,x) -> b .= Φᴹ .* x, backw = (b,x) -> b .= x)
y = Φ * Xᴳᵀ
@show rank(y);

size(Xᴳᵀ) = (60, 40)
rank(Xᴳᵀ) = 18
rank(y) = 40


In [7]:
@time RPCA_IRLS(Xᴳᵀ, y, Φ, N = 20, r̃ = r, s̃ = s, λ = 1, μ = 0.5, δ = 0.5, verbose = true);

k = 0,	rank(L) = 40,	‖y - Φ * X‖₂ =   0.000,	‖Xᴳᵀ - X‖₂ = 120.816,	σ₁ =  43.166,	ϵ =     Inf
	|S|₍ₛ₊₁₎/(2λ) =   0.863, σᵣ₊₁ =   3.512
k = 1,	rank(L) = 40,	‖y - Φ * X‖₂ =  38.540,	‖Xᴳᵀ - X‖₂ = 126.814,	σ₁ =  10.792,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) =   0.252, σᵣ₊₁ =  16.211
k = 2,	rank(L) = 40,	‖y - Φ * X‖₂ =   6.990,	‖Xᴳᵀ - X‖₂ = 145.549,	σ₁ =  84.028,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) = 0.00925, σᵣ₊₁ = 224.142
k = 3,	rank(L) = 27,	‖y - Φ * X‖₂ =   1.079,	‖Xᴳᵀ - X‖₂ = 1398.945,	σ₁ = 859.507,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) =  0.0085, σᵣ₊₁ = 107.791
k = 4,	rank(L) = 27,	‖y - Φ * X‖₂ =   1.119,	‖Xᴳᵀ - X‖₂ = 745.841,	σ₁ = 472.684,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) = 0.00455, σᵣ₊₁ =  77.054
k = 5,	rank(L) = 27,	‖y - Φ * X‖₂ =   0.588,	‖Xᴳᵀ - X‖₂ = 409.986,	σ₁ = 199.107,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) = 0.00308, σᵣ₊₁ =  93.564
k = 6,	rank(L) = 27,	‖y - Φ * X‖₂ =   0.409,	‖Xᴳᵀ - X‖₂ = 590.129,	σ₁ = 426.752,	ϵ =   3.512
	|S|₍ₛ₊₁₎/(2λ) = 0.00351, σᵣ₊₁ =  84.266
k = 7,	rank(L) = 27,	‖y - Φ * X‖₂ =   0.466,	‖Xᴳᵀ - X‖₂ = 525